In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
#from ipywidgets import interact, interactive
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
import ipywidgets as widgets
import asyncio

In [3]:
iOption0 = 1
part = 0
N_loc_files = 2

In [4]:
N_centr = 5
N_pt = 1
#colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,1.5,1.0,2.0,0.5,5.0,1.0,5.0,0.5,5]
colors = [3,1,2,4,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_names=["../QA/input/NewHadron/missalign_hadron_new8_rg0-8.root","../sim/output/embed/piminus_10M_embed_hagedorn_v0_ddhit.root"]
part_names = ["b#bar{b}","c#bar{c}","#phi","J/#psi","#gamma#to ee"]
hist_dphi_names = ["dphi_hist_el_dynamic_2"]
hist_dthe_names = ["dthe_hist_el_dynamic_2"]
hist_sdphi_names=["sdphi_hist_el_dynamic_2"]
hist_sdthe_names=["sdthe_hist_el_dynamic_2"]
hists_all = [hist_dphi_names,hist_dthe_names,hist_sdphi_names,hist_sdthe_names]
N_type = len(hists_all)

In [5]:
hists_read = []

N_files = len(file_names)
for iFile in range(N_files):
    infile = root.TFile.Open(file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hist_select_3D0 = []
        for ihist in range(len(hists_all[itype])):
            hist_select_3D0.append(infile.Get(hists_all[itype][ihist]))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            hist_select_3D0[-1].SetName(hist_select_3D0[-1].GetName()+f"{iFile}")
        hists_emc_read0.append(hist_select_3D0)
    hists_read.append(hists_emc_read0)

infile.Close()

In [6]:
legends, titles, leftline, rightline = [], [], [], []
def pair_DCA_draw(clock, x = 0.5, y = 1.5, phithe = 0, data = 0, iter = 0):
    ipt = 2
    type_loc =phithe
    c0 = root.TCanvas(f"c2_{clock}",f"c2_{clock}",1400,450*1)
    c0.Divide(2,1)
    N_dca_files = 1
    for itype in range(1):
        max = 0

        if iter == 1 : 
            legends.append(root.TLegend(0.75,0.6,0.95,0.95))
            Format_Legend(legends[-1],0.03)
            titles.append(Draw_Legend_Title(0.3,0.85,0.75,0.95,"Au+Au mb sim",0.075))
        for iFile in range(0+data,N_dca_files+data):
            c0.cd(itype+1)
            ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
            ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
            ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
            ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
            hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibinZ0,ibinZ1)
            project1= hists_read[iFile][type_loc][itype].Project3D("yx")
            clr = colors[iFile]
            Format_Hist_total(project1, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15,  0.01, 0.01,  .07,  0.06, 21,  0, clr,  7,  clr,  1, 1.1, "",  1,  1-0.15*iFile)
            if iFile == data :
                project1.Draw("colz")
                max = project1.GetMaximum()
            else:
                project1.Scale(max/project1.GetMaximum())          
                project1.Draw("H same")
            if iter == 1 :
                leftline.append(root.TF1(f"lefty_{iFile}_{itype}_{clock}_{iter}","((x<[1])?pow(2,20)*(x-[0]):pow(2,20)*(x-0.6))",-0.1,0.1))
                rightline.append(root.TF1(f"right_{iFile}_{itype}_{clock}_{iter}","pow(2,20)*(x-[0])",-0.1,0.1))
                Format_Func(leftline[-1], 3,root.kViolet,1,0.5)
                leftline[-1].SetFillColor(root.kViolet)
                leftline[-1].SetFillStyle(3001)
                Format_Func(rightline[-1],3,root.kViolet,1,0.5)
            leftline[-1].SetParameter(0,x)
            leftline[-1].SetParameter(1,y)
            leftline[-1].Draw("E3 same")
            rightline[-1].SetParameter(0,y)
            #rightline[-1].Draw("same")


            c0.cd(itype+2)
            ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(x)
            ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(y)
            ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
            ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
            project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+f"_X{iter}",ibinY0,ibinY1,ibinZ0,ibinZ1)
            clr = colors[iFile]
            Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15,  0.01, 0.01,  .07,  0.06, 21,  0, clr,  7,  clr,  1, 1.1, "",  1,  1-0.15*iFile)
            #project.GetXaxis().SetRange(project.FindBin(1),project.FindBin(1100))
            #root.gPad.SetLogy()
            if iFile == data :
                project.Draw("H")
                max = project.GetMaximum()
            else:
                project.Scale(max/project.GetMaximum())          
                project.Draw("H same")
            if iter == 1 : legends[-1].AddEntry(project,part_names[iFile],"l")
        legends[itype].Draw()
        titles[itype].Draw()


    c0.Draw()
    #c0.SaveAs(f"output/dca2.png")

    return c0

In [7]:
if False:
    c = pair_DCA_draw(1,-0.005,-0.001,0,0,1)
    c.Draw()

In [8]:
clock = 0

In [11]:

clock += 1

slider2 = widgets.FloatRangeSlider(
    value=[-0.01, 0.01],
    min=-0.05,
    max= 0.05,
    step=0.001,
    description='Mass [GeV]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=widgets.Layout(width='75%'),
)

async def test2(slider2, output, phithe=0, data = 0):
    i = 0
    while slider2.value[1]< 0.049 and slider2.value[0] < 0.049:
        i = i+1
        await asyncio.sleep(1)
        output.update('test2 ' + str(slider2.value) + ' - ' + str(i))
        c1= pair_DCA_draw(clock, slider2.value[0],slider2.value[1],phithe, data, i)
        if i == 1: c1.Draw()
        else : c1.Update()
    return

output = display("tbd", display_id=True)
display(slider2)


task2 = asyncio.create_task(test2(slider2, output, 0, 0))

'test2 (0.044, 0.05) - 28'

FloatRangeSlider(value=(-0.01, 0.01), continuous_update=False, description='Mass [GeV]:', layout=Layout(width=…

In [10]:
if True: was_cancelled = task2.cancel()

import ipywidgets as widgets
import asyncio

slider = widgets.IntSlider(
    value=5,
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

async def test(slider, output):
    i = 0
    while slider.value != 10:
        i = i+1
        await asyncio.sleep(1)
        output.update('test ' + str(slider.value) + ' - ' + str(i))
        x = slider.value 
        c1= single_DCA_draw(slider.value)[0]
        if i == 1: c1.Draw()
        else : c1.Update()

output = display("tbd", display_id=True)
display(slider)

#asyncio.create_task(test(slider, output))